# Unsupervised learning on Weblogs v2

> This Notebook is part of a study on applied machine learning for quick incident response, so please cite the "Taking advantage of unsupervised learning in incident response" article published in IWSSS 2020 proceedings -- "2020 8th International Workshop on Systems Safety & Security (IWSSS)" and "Machine Learning Approach to Quick Incident Response" article published in COMM2020 proceedings -- "2020 13th International Conference on Communications (COMM)".

## 0. Intro

### Creating a dataset from unseen before weblogs 

The initial tests were conducted on weblogs that had reported incidents, to test the applicability of unsupervised learning techniques. 
One of the characteristics of incident collected web access logs is their randomness. The events would be different from case to case and would contain seen before attack patterns, and more often than not, unseen ones. Considering this fact, the development of a rule-based tool would be extremely complicated, and could also miss relevant data. 
Employing machine learning methods could result in improved results on new unseen data, compared to standard incident response tools and techniques.
Nevertheless, the extrapolated dataset will not respect best practice norms like:
unique records for the training and testing, so that frequency will not influence the performance of the model;
properly labeled data;
an equal number of records to represent each class in the observed space;
standard configuration and network architecture so that the result can be duplicated. 
Unsupervised learning could represent the answer, but as seen in earlier attempts, clustering on weblogs extracted features is not that easy. 
For this part of this study, we used publicly available web access logs from the following sources: [Try Google query inurl:access.log filetype:log]
- http://www.almhuette-raith.at/apache-log/access.log
- https://www.todaynewsportal.com/access.log

Features to look after:
-    unusual_hours;
-    has_bad_rep;
-    shannon_entropy;      #new
-    has_bad_string
-    method;
-    version;
-    status;
-    log_len_uri; 
-    log_bytes;
-    scripting_useragent;
-    label;                #Binary 1=Suspicious and 0=Safe
-    id.                   #new

In [ ]:
import pandas as pd
df = pd.read_csv("../input/10k-idlabel/_dataset_web_10K_id_2_.csv")
df.describe()

In [ ]:
df.drop(["id"],axis=1,inplace=True)    # dropped

df.tail()   # last 5 rows

### Evaluation of the dataset

Using cross-validation with ten folds method, we evaluated the efficiency of different machine learning algorithms on the resulting dataset.
To provide us with a quick start, we conducted the experiments with default parameters in Weka 3.8.4.

Algorithm | Precision |    MAE     | Obs.
---|---|---|---
ZeroR|50.15 %|0.50|baseline
OneR|96.91 %|0.03|used just shannon_entropy to make the prediction
NaiveBayes|71.29 %|0.30|
SVM|74.40 %|0.26|    
J48|99.18 %|0.01|
RandomForest|99.37 %|0.01|most accurate on extracted features

### Exploratory data analysis (EDA)

EDA is an approach to dataset analysis in which the data is summarized using its main characteristics, often in visual form. We observe that the suspicious events are not easily separable from the safe ones, highlighted by the two different colors corresponding to the safe and suspicious labels.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

sns.pairplot(data=df,hue="label",palette="Set2")
plt.show()

In [ ]:
import matplotlib#.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D 

fig = plt.figure(figsize=(10, 10))
#fig = plt.figure()

ax = Axes3D(fig)
colors = ['red','green']
'''
shannon_entropy
log_bytes
scripting_useragent
'''
xs = df['status']
ys = df['log_len_uri']
zs = df['log_bytes']
#mapping = {'safe': 0, 'suspicious': 1}
label = df['label']#.replace(mapping)
ax.scatter(xs, ys, zs, c=label, cmap=matplotlib.colors.ListedColormap(colors), edgecolors='w')

ax.set_xlabel('status')
ax.set_ylabel('log_len_uri')
ax.set_zlabel('log_bytes')

plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

features = ['unusual_hours','has_bad_rep','shannon_entropy','method','version','status','log_len_uri','log_bytes','scripting_useragent']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
#df['label'].replace(0, 'Safe',inplace=True)
#df['label'].replace(1, 'Suspicious',inplace=True)
y = df.loc[:,['label']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
finalDf = pd.concat([principalDf, df[['label']]], axis = 1)

fig = plt.figure(figsize=(10, 10))
ax = Axes3D(fig)
colors = ['r', 'g', 'b']

ax.set_xlabel('principal component 1', fontsize = 15)
ax.set_ylabel('principal component 2', fontsize = 15)
ax.set_zlabel('principal component 3', fontsize = 15)

ax.set_title('3 component PCA', fontsize = 20)

targets = [0, 1]

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['label'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , finalDf.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 50)

ax.legend(targets)
ax.grid()
plt.show()

# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
finalDf['label'].replace(0, 'Safe',inplace=True)
finalDf['label'].replace(1, 'Suspicious',inplace=True)

safe=finalDf.loc[finalDf['label'] == 'Safe']
suspicious=finalDf.loc[finalDf['label'] == 'Suspicious']

finalDf['label'].replace(0, 'Safe',inplace=True)
finalDf['label'].replace(1, 'Suspicious',inplace=True)


trace1 = go.Scatter3d(
    x=safe['principal component 1'],
    y=safe['principal component 2'],
    z=safe['principal component 3'],
    mode='markers',
    name = "Safe",
    marker=dict(
        size=12,
        color='rgba(66, 135, 245, 0.8)'
    ),
    text= finalDf.label
)

trace2 = go.Scatter3d(
    x=suspicious['principal component 1'],
    y=suspicious['principal component 2'],
    z=suspicious['principal component 3'],
    mode='markers',
    name = "Suspicious",
    marker=dict(
        size=12,
        color = 'rgba(235, 64, 52, 0.8)'
    ),
    text= finalDf.label
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace1, trace2]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

## 1. Testing K-Means Clustering 

### Adjusting the dataset for unsupervised learning
For unsupervised learning, we do not need the labels for the feature vectors. We are not dropping the scripting_useragent column, even though the value is always 0 since, in the new data, we could have other values. This characteristic can be quickly dropped in the code below as needed. We also encode the labels as 0 for safe items and 1 for suspicious.

In [ ]:
from scipy.stats import entropy 
from pandas.core.algorithms import value_counts

#X = df.drop(['label'], axis=1)
#mapping = {'safe': 0, 'suspicious': 1}
#y = df['label'].replace(mapping)

### Finding the ideal number of clusters

Ideally, the dataset should be grouped around two cluster centers and should be easily separated into the two categories, safe and suspicious. If we use methods for ideal cluster number determination, like the ones presented below, we observe that the theoretical number of clusters is way beyond two.
To demonstrate, we use SciKit-Learn implementation with most of the parameters as default. Even though we already know that we should differentiate two clusters, it is an outstanding practice to check the ideal amount with algorithms like: [1,2,3,4]
- The elbow method looks at the total With-in-Sum-of-Squares (WSS) as a function of the number of clusters. We should choose a number of clusters so that the addition of one more will not significantly improve the total WSS;
- The gap statistic method compares the total within intra-cluster variation for different k’s with their expected values under the null reference distribution of the data. The optimal k will be the value that maximizes the gap statistic (e.g., that yields the most significant gap statistic). (R. Tibshirani, G. Walther, and T. Hastie, Standford University, 2001);
- The Silhouette method computes the average silhouette of observations for different values of k. The optimal k maximizes the average silhouette (Kaufman and Rousseeuw 1990).

In [ ]:
##Calculation 
%matplotlib inline
import time
import hashlib
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import silhouette_score

def optimalK(data, nrefs=3, maxClusters=20):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal

sil = []
kmax = 20
wcss = []

for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0).fit(X)
  labels = kmeans.labels_
  sil.append(silhouette_score(X, labels, metric = 'euclidean'))

for k in range(1,kmax):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
k, gapdf = optimalK(X, nrefs=5)
#print('Optimal k (GAP): ', k)

In [ ]:
##Visualisation 
plt.figure(figsize=(20,10))
plt.suptitle('Ideal K',fontsize=20)

plt.subplot(1,2,1)
plt.plot(range(1,20),wcss,"-o")
plt.grid(True)
plt.ylabel('WCSS',fontsize=35)
'''
plt.subplot(1,3,2)
plt.plot(gapdf.clusterCount, gapdf.gap, linewidth=3)
plt.grid(True)
plt.scatter(gapdf[gapdf.clusterCount == k].clusterCount, gapdf[gapdf.clusterCount == k].gap, s=250, c='r')
plt.xlabel('Cluster Count',fontsize=20)
plt.ylabel('Gap Value',fontsize=35)

plt.subplot(1,3,3)
plt.plot(range(1,20),sil)
plt.grid(True)
plt.ylabel("Silhouette Score",fontsize=35)
'''
x_ticks=['x tick 1','x tick 2','x tick 3']
ax.set_xticklabels(x_ticks, rotation=0, fontsize=3)

plt.show()

import pandas as pd
import numpy as np

X = df.drop(['label'], axis=1)
#mapping = {'safe': 0, 'suspicious': 1}
y = df['label']#.replace(mapping)

#data = df
#X = data.iloc[:,0:20]  #independent columns
#y = data.iloc[:,-1]    #target column i.e price range
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

**As seen above, the results obtained from the elbow and gap methods are not promising, since they return very high values for k. Nevertheless, the third evaluation is encouraging since the Silhouette Score is at its maximum when k is 3. To summarise, none of the tests gave us the desired k = 2. This result is an expected one since there are situations when the extracted features have very close values for both safe and suspicious events. The next step should be to enhance somehow the dataset so that the clusters could be more separable.**

## 2. Dataset enrichment

As in hybrid classifiers, we intend to use additional supervised algorithms to improve the overall results.
First, we tried Linear Regression and evaluated the dataset based on the following primary metrics:
- Mean Absolute Error (MAE) – the average error;
- Mean Squared Error (MSE) – more significant errors are punished;
- Root Mean Squared Error (RMSE) – it is recommended that it should be used as the primary metric to interpret your model.

In [ ]:
##Linear Regression
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
from sklearn.metrics import precision_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
from sklearn import metrics
import joblib

features = df.drop(['label'], axis=1)

train = df.sample(n=900, replace=True, random_state=1)
test = df.drop(train.index)

X_train = train.drop(['label'], axis=1)
mapping = {'safe': 0, 'suspicious': 1}
y_train = train['label'].replace(mapping)

X_test = test.drop(['label'], axis=1)
mapping = {'safe': 0, 'suspicious': 1}
y_test = test['label'].replace(mapping)

regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

#To retrieve the intercept:
print('Intercept: %.8f' % regressor.intercept_)
#For retrieving the slope:
#print(regressor.coef_)

# save the model to disk
filename = 'LR.model'
joblib.dump(regressor, filename)

y_pred = regressor.predict(X_test).round()

y_test = np.array(list(y_test))
y_pred = np.array(y_pred)

count=0

for i in range(len(y_test)):
    if y_test.flatten()[i] !=  round(y_pred.flatten()[i]):
        count+=1
     
precision = precision_score(y_test, y_pred, average='micro')
print('Precision: %.2f' % float(precision*100), "%")

#print(count)
#print(len(y_test))

#dataset2 = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
#dataset2

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
coeffecients = pd.DataFrame(regressor.coef_,X_test.columns)
coeffecients.columns = ['Coeffecient']
#coeffecients

**The precision on the observed known data has improved significantly. Let us try adding one more feature based on the original ones by employing the Multi-layer Perceptron algorithm.**

In [ ]:
##Multi-layer Perceptron 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score
import joblib

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X_train, y_train)
#print([coef.shape for coef in clf.coefs_])
#print(clf.coefs_)

y_pred = clf.predict(X_test)

precision = precision_score(y_test, y_pred, average='micro')
print('Precision: %.2f' % float(precision*100), "%")

features["mlp_pred"] = (clf.predict(features[["unusual_hours", 
                                             "has_bad_rep",
                                             "has_bad_string", 
                                             "method", 
                                             "version", 
                                             "status", 
                                             "log_len_uri", 
                                             "log_bytes", 
                                             "scripting_useragent"]]))

# save the model to disk
filename = 'MLP.model'
joblib.dump(clf, filename)

In [ ]:
features["lr_pred"] = (0.13688273 * df["unusual_hours"] 
                     + 0.03655747 * df["has_bad_rep"] 
                     + 0.04017277 * df["has_bad_string"] 
                     - 0.05029479 * df["method"] 
                     + 0.34400104 * df["version"] 
                     + 0.20885592 * df["status"] 
                     + 0.41214625 * df["log_len_uri"]
                     - 0.00410462 * df["log_bytes"]
                     + 0.00000000 * df["scripting_useragent"]
                     - 0.45363860)

df["lr_pred"] = (0.13688273 * df["unusual_hours"] 
                     + 0.03655747 * df["has_bad_rep"] 
                     + 0.04017277 * df["has_bad_string"] 
                     - 0.05029479 * df["method"] 
                     + 0.34400104 * df["version"] 
                     + 0.20885592 * df["status"] 
                     + 0.41214625 * df["log_len_uri"]
                     - 0.00410462 * df["log_bytes"]
                     + 0.00000000 * df["scripting_useragent"]
                     - 0.45363860)

df["mlp_pred"] = (clf.predict(df[["unusual_hours", 
                                    "has_bad_rep",
                                    "has_bad_string", 
                                    "method", 
                                    "version", 
                                    "status", 
                                    "log_len_uri", 
                                    "log_bytes", 
                                    "scripting_useragent"]]))

## 3. Testing unsupervised learning on the new dataset

We have plotted the graphical representation based on the most significant features (log_bytes and has_bad_strings) for k between 1 and 4, as suggested by the CfsSubsetEval algorithm [5].

In [ ]:
plt.figure(figsize=(24,4))

plt.suptitle("K Means Clustering",fontsize=20)

plt.subplot(1,5,1)
plt.title("K = 1",fontsize=16)
plt.xlabel("log_len_uri")
plt.ylabel("has_bad_strings")
plt.scatter(features.log_len_uri,features.has_bad_string)


plt.subplot(1,5,2)
plt.title("K = 2",fontsize=16)
plt.xlabel("log_len_uri")
kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
features["labels"] = kmeans.fit_predict(features)
plt.scatter(features.log_len_uri[features.labels == 0],features.has_bad_string[features.labels == 0])
plt.scatter(features.log_len_uri[features.labels == 1],features.has_bad_string[features.labels == 1])

# I drop labels since we only want to use features.
features.drop(["labels"],axis=1,inplace=True)

plt.subplot(1,5,4)
plt.title("K = 3",fontsize=16)
plt.xlabel("log_len_uri")
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
features["labels"] = kmeans.fit_predict(features)
plt.scatter(features.log_len_uri[features.labels == 0],features.has_bad_string[features.labels == 0])
plt.scatter(features.log_len_uri[features.labels == 1],features.has_bad_string[features.labels == 1])
plt.scatter(features.log_len_uri[features.labels == 2],features.has_bad_string[features.labels == 2])

# I drop labels since we only want to use features.
features.drop(["labels"],axis=1,inplace=True)

plt.subplot(1,5,3)
plt.title("K = 4",fontsize=16)
plt.xlabel("log_len_uri")
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
features["labels"] = kmeans.fit_predict(features)
plt.scatter(features.log_len_uri[features.labels == 0],features.has_bad_string[features.labels == 0])
plt.scatter(features.log_len_uri[features.labels == 1],features.has_bad_string[features.labels == 1])
plt.scatter(features.log_len_uri[features.labels == 2],features.has_bad_string[features.labels == 2])
plt.scatter(features.log_len_uri[features.labels == 3],features.has_bad_string[features.labels == 3])

# I drop labels since we only want to use features.
features.drop(["labels"],axis=1,inplace=True)

plt.subplot(1,5,5)
plt.title("Original Labels",fontsize=16)
plt.xlabel("log_len_uri")
plt.scatter(df.log_len_uri[df.label == "suspicious"],df.has_bad_string[df.label == "suspicious"])
plt.scatter(df.log_len_uri[df.label == "safe"],df.has_bad_string[df.label == "safe"])

plt.subplots_adjust(top=0.8)
plt.show()

**We observe that for k = 2, the graphical representation is quite similar to the original labeling, but we still do not observe separable clusters.**

**Let us also try the hierarchical clustering method. We are looking for the longest vertical line without any perpendicular matching lines (euclidian distances). We observed three significant groups in the graphical representation of the hierarchical clustering method on our data, so the ideal number of labels would be three.**

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

merg = linkage(features,method="ward")

plt.figure(figsize=(18,6))
dendrogram(merg, leaf_rotation=90)
plt.xlabel("data points")
plt.ylabel("euclidian distance")

plt.suptitle("DENDROGRAM",fontsize=18)
plt.show()

**This result is a more realistic depiction of our data since the dataset contains elements with almost identical feature vectors from distinct classes.**

## 4. Evaluation and comparison of the results

In [ ]:
from sklearn.cluster import AgglomerativeClustering

kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
features["labels"] = kmeans.fit_predict(features[['lr_pred', 'mlp_pred']])

# cross tabulation table for kmeans
df1 = pd.DataFrame({'pred':features['labels'] ,"label":df['label'] })
ct1 = pd.crosstab(df1['label'],df1['pred'])


# hierarchy
hc_cluster = AgglomerativeClustering(n_clusters=2)
hc_predict = hc_cluster.fit_predict(features[['lr_pred', 'mlp_pred']])

# cross tabulation table for Hierarchy
df2 = pd.DataFrame({'labels':hc_predict,"label":df['label']})
ct2 = pd.crosstab(df2['labels'],df2['label'])


plt.figure(figsize=(24,8))
plt.suptitle("CROSS TABULATIONS",fontsize=18)
plt.subplot(1,2,1)
plt.title("KMeans")
sns.heatmap(ct1,annot=True,cbar=False,cmap="Blues")

plt.subplot(1,2,2)
plt.title("Hierarchy")
sns.heatmap(ct2,annot=True,cbar=False,cmap="Blues")

plt.show()

The routine below will evaluate the 10k extracted dataset, from the original web access log. You can use the extractor script to <extract> features from the rest of the web logs.

In [ ]:
import pandas as pd
import joblib

d10k = pd.read_csv("../input/10k-access/dataset_web_10K.csv", delimiter=',')

# load the models from disk
MLP_model = joblib.load('MLP.model')
LR_model = joblib.load('LR.model')

#print(d10k[['scripting_useragent']])
d10k["mlp_pred"] = (MLP_model.predict(d10k[["unusual_hours", 
                                                 "has_bad_rep",
                                                 "has_bad_string", 
                                                 "method", 
                                                 "version", 
                                                 "status", 
                                                 "log_len_uri", 
                                                 "log_bytes", 
                                                 "scripting_useragent"]]))

d10k["lr_pred"] = (LR_model.predict(d10k[["unusual_hours", 
                                                 "has_bad_rep",
                                                 "has_bad_string", 
                                                 "method", 
                                                 "version", 
                                                 "status", 
                                                 "log_len_uri", 
                                                 "log_bytes", 
                                                 "scripting_useragent"]]))

kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
d10k["labels"] = kmeans.fit_predict(d10k[['lr_pred', 'mlp_pred']])

plt.figure(figsize=(15,10))
plt.title("Unknown items", fontsize=18)
plt.grid(True)
plt.scatter(d10k.log_len_uri[d10k.labels == 0],d10k.lr_pred[d10k.labels == 0])
plt.scatter(d10k.log_len_uri[d10k.labels == 1],d10k.lr_pred[d10k.labels == 1])
plt.xlabel("log_len_uri",fontsize=14)
plt.ylabel("lr_pred",fontsize=14)
plt.tight_layout()
plt.show()

d10k.to_csv (r'export_labeled_df.csv', index = False, header=True)
print("Saved csv!")

In [ ]:
import matplotlib#.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D 

fig = plt.figure(figsize=(10, 10))
#fig = plt.figure()

ax = Axes3D(fig)
colors = ['red','green']

xs = d10k['status']
ys = d10k['log_len_uri']
zs = d10k['log_bytes']
label = d10k.labels
ax.scatter(xs, ys, zs, c=label, cmap=matplotlib.colors.ListedColormap(colors), edgecolors='w')

ax.set_xlabel('status')
ax.set_ylabel('log_len_uri')
ax.set_zlabel('log_bytes')

plt.show()

# Conclusion

### The features extracted from the weblogs are not enough for clustering, so we need to implement supervised learning algorithms to extract data form labeled entries. 

### As future work, the analysis process applied to requests can be improved, switching from a string match method to a principal component analysis (PCA) and entropy calculation.

# Sources

[1] Satsawat Natakarnkitkul, Get the Optimal K in K-Means Clustering, https://medium.com/towards-artificial-intelligence/get-the-optimal-k-in-k-means-clustering-d45b5b8a4315

[2] Robert Tibshirani, Guenther Walther and Trevor Hastie, Estimating the number of clusters in a data set via the gap statistic, https://statweb.stanford.edu/~gwalther/gap

[3] https://www.programcreek.com/python/example/97803/tqdm.tqdm_notebook

[4] Khyati Mahendru, How to Determine the Optimal K for K-Means?, https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb

[5] M. A. Hall, Correlation-based Feature Subset Selection for Machine Learning. Hamilton, New Zealand (1998).

[6] Efe Ergun, Unsupervised Learning on Iris, https://www.kaggle.com/efeergun96/unsupervised-learning-on-iris